## QAOA
QAOAはVQEと同様で特定のansatzを組み合わせ最適化問題に使います。
量子断熱発展は始点から終点まで基底状態を保ちます。

現時刻をt、全体のスケジュールをTとし、初期のハミルトニアンを $H_{start}$、最後のハミルトニアンを $H_{final}$ とします。

$$
H_{temp} = (1-\frac{t}{T})H_{start} + \frac{t}{T}H_{final}
$$


$T\rightarrow\infty$ のとき、基底状態は以下のような固有状態になります。

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

時間発展は

$$
\mid \psi_{t+1} \rangle = U \mid \psi_t \rangle = e^{-iHt}  \mid \psi_t \rangle
$$

回路は

![https___qiita-user-contents.imgix.net_https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fe10f1843-cc16-cdfe-e4a6-e2fbaab6df9f.png_ixlib=rb-1.2.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/812819f5-0037-5a30-2690-8ed39c0b1a06.png)

1. 状態の設定
2. qaoa ansatz

の2つの部分にわかれています。

qaoa ansatz はさらに2つの部分にわかれていて、

1. cost ハミルトニアン
2. mixer ハミルトニアン

上の例では、Hゲートが設定の部分でその他が ansatz 部分です。

ansatz に関しては,
CX-Rz-CX-Rz が cost ハミルトニアン で Rx が mixer ハミルトニアンです。

cost ハミルトニアンは細かく述べると, CX-Rz-CX の部分が重みの部分で Rz が バイアスの部分です.

ここで標準状態として |+> を準備し、 HゲートとXゲート(時間発展させるためRxに変換させます。)を mixer として使います。 

簡単な問題をしてみます。

In [1]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

h = 2*q(1)-4*q(0)*q(1)
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((1, 1), 0.9898604494842518), ((1, 0), 0.006673870363175068), ((0, 0), 0.0029313019700286922), ((0, 1), 0.0005343781825440884))


求める解のビットの組み合わせと確率振幅から確率が得られます。

## QAOAを用いた交通最適化
もう一つの例としてフォルクスワーゲン社の交通流最適を参考にしてみましょう。

「Quantum Computing at Volkswagen:
Traffic Flow Optimization using the D-Wave Quantum Annealer」
引用：https://www.dwavesys.com/sites/default/files/VW.pdf

ステップは以下のようになります、

1. 各車における経路をそれぞれ2つ決める。 (古典)
2. 混雑量を計算する。 (古典)
3. 1で決めた各車のルートを変えて混雑量を最適化させる。 (QAOA)

まずスタート地点をA、ゴール地点をBとして、各経路に0から11まで番号を振っています。
各車それぞれ2つずつ経路を考え、混雑量を最適化させることを目標とします。

![0_jdvbeWCvUMG-UKis.jpeg](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/d401be2a-5971-ca4f-c00e-63403372bae8.jpeg)

car1  
route1-1(q0)：s0,s3,s6,s9  
route1–2 (q1)：s0,s3,s8,s11  

car2  
route2–1(q2):s0,s3,s8,s11  
route2–2(q3):s2,s7,s10,s11  

## Mixerを選ぶ
提案した2つの経路から1つだけ選ぶので、片方は1でもう片方は0となります。
これらの経路を量子ビットで考えるなら $q_0$ と $q_1$ のどちらかは 1 で もう１つは 0 となります。 $q_2$ と $q_3$ も同様に考えます。

上記の内容について量子もつれを用います。

In [2]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].m[:].run(shots=100)

Counter({'00': 50, '11': 50})

この回路は |00> と |11> のもつれを作ります。
また Xゲートを使うことで、

In [3]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].m[:].run(shots=100)

#=>Counter({'01': 41, '10': 59})

Counter({'10': 45, '01': 55})

これを使うことで探索空間を制限させることができます。   
この量子もつれを各量子ビットに用いることで、以下のように2つもつれを作ることができます。

In [4]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2].m[:].run(shots=100)

Counter({'1001': 26, '1010': 23, '0110': 31, '0101': 20})

## cost 関数の混雑量
混雑量を計算してみましょう。経路を以下のように考えます。

car1  
route1-1(q0)：s0,s3,s6,s9  
route1–2 (q1)：s0,s3,s8,s11  

car2  
route2–1(q2):s0,s3,s8,s11  
route2–2(q3):s2,s7,s10,s11  

考えた経路が何回現れたか見ることで、混雑量を計算できます。

s0 から s11 までの経路から、全ての経路の混雑量を2乗して計算することができます。

$$
h=(q_0+q_1+q_2)^2+(q_0+q_1+q_2)^2+q_0^2+q_0^2+(q_1+q_2)^2+(q_1+q_2+q_3)^2+q_3^2+q_3^2+q_3^2\\
=4q_0+4q_1+4q_2+4q_3+4q_0q_1+4q_0q_2+8q_1q_2+2q_1q_3+2q_2q_3
$$

ここで, $0^2=0,1^2=1$ を用いた。

これが cost ハミルトニアン です。

In [8]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

def an(index):
    return 0.5 * X[index] + 0.5j * Y[index]

def cr(index):
    return 0.5 * X[index] - 0.5j * Y[index]

op1 = (cr(1) * an(0) + cr(0) * an(1)).to_expr().simplify()
op2 = (cr(3) * an(2) + cr(2) * an(3)).to_expr().simplify()

class QaoaQaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]
        self.time_evolutions1 = [term.get_time_evolution() for term in op1]
        self.time_evolutions2 = [term.get_time_evolution() for term in op2]
        
    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            for evo1 in self.time_evolutions1:
                evo1(c, beta)
            for evo2 in self.time_evolutions2:
                evo2(c, beta)
        return c

h = 4*q(0)+4*q(1)+4*q(2)+4*q(3)+4*q(0)*q(1)+4*q(0)*q(2)+8*q(1)*q(2)+2*q(1)*q(3)+2*q(2)*q(3)

runner = Vqe(QaoaQaoaAnsatz(h.to_expr().simplify(),4,Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2]))
result = runner.run()

# get probability
print(result.most_common(12))

(((1, 0, 0, 1), 0.5678750608374458), ((0, 1, 0, 1), 0.358480123549911), ((1, 0, 1, 0), 0.06843852251604825), ((0, 1, 1, 0), 0.005206293096588184), ((0, 1, 1, 1), 2.3296831016347912e-31), ((1, 1, 0, 1), 1.2330635659635428e-31), ((1, 0, 0, 0), 6.162975822039155e-32), ((0, 1, 0, 0), 4.005934284325449e-32), ((0, 0, 1, 0), 3.7623996265149723e-32), ((0, 0, 0, 1), 2.1128904100813353e-32), ((1, 1, 1, 0), 1.0846701092368921e-32), ((1, 0, 1, 1), 9.870390964984581e-33))


## 古典最適化部分
最適化は古典コンピュータの最適化アルゴリズムのよって行われます。

In [ ]:
minimizer=vqe.get_scipy_minimizer(method="COBYLA",options={"tol":5.0e-4})
runner = Vqe(QubitAnsatz(cost),minimizer=minimizer)
result = runner.run()

blueqat では scipy から optimizer を選択できます。

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

'Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP'

もし独自の optimizer を用いる場合、 blueqat での vqe 用のベイズオプティマイザである "hyperopt" を使いましょう。

In [5]:
!pip install hyperopt

     |████████████████████████████████| 972kB 888kB/s 
  Using cached https://files.pythonhosted.org/packages/41/8f/dd6a8e85946def36e4f2c69c84219af0fa5e832b018c970e92f2ad337e45/networkx-2.4-py3-none-any.whl
     |████████████████████████████████| 829kB 4.5MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491057 sha256=50eec7093ce883ffa5bf635b3f1c83985bd35b429aff5cd5a369d600d60c703d
  Stored in directory: /Users/kei/Library/Caches/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


とりあえず関数を作ります。

def hyperopt_minimizer(objective, n_params):
    from hyperopt import fmin, Trials, tpe, hp
    trials = Trials()
    best = fmin(objective, [hp.uniform(f'p{i}', 0., 2 * np.pi) for i in range(n_params)],
            algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)
    return list(best.values())
```

設定をすることで使えます。

```python
h = 2*q(1)-4*q(0)*q(1)

runner = Vqe(QubitAnsatz(h),minimizer=hyperopt_minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))
```

```
100%|██████████| 100/100 [00:00<00:00, 184.33it/s, best loss: -1.9999826942162187]
Result by VQE
-1.9999826942162187
```